In [1]:


import pyomo.environ as pyo
import pandas as pd

import sqlite3

cnx = sqlite3.connect('/Users/kultiginbozdemir/Documents/GitHub/op_room_opt/WebApp/test.db') # change the path


def get_ops(date):
    # E.g. we will have 365 tables, named with name of coresponding dates. 
    query = 'SELECT * FROM {}'.format(date) # change name of the table.  save them with date in db. 
    basket = pd.read_sql_query(query, cnx)
    
    # DEp_capacity and op_capacity tables have 365 columns refer each day 
    query2 = 'SELECT {} FROM dep_cap'.format(date)
    dep_df = pd.read_sql_query(query2, cnx)# departments
    op_df = pd.read_sql_query(query2, cnx)# operations room
    return (basket, dep_df,op_df)


basket,dep_df,op_df=get_ops(user__entries)
# We need to add the last entry into basket dataframe.
basket.loc[id,('ops', 'dep_id','operation_duration','operation_urgency')]= # (xx,xx,xx,xx) from your query
    ##################

# multi-knapsack, integer divisible

mdl = pyo.ConcreteModel()

# sets
mdl.invs = pyo.Set(initialize=list(zip(basket.index, basket["dep_id"])))
mdl.bins = pyo.Set(initialize=list(op_df.index)) ## list of operations room from db
mdl.deps = pyo.Set(initialize=list(dep_df.index)) # list of departments from db

# params
mdl.value   = pyo.Param(mdl.invs, initialize= {(i,row["dep_id"]):row["op_urgency"] for i,row in basket.iterrows()} )
mdl.weight  = pyo.Param(mdl.invs, initialize= {(i,row["dep_id"]):row["op_duration"] for i,row in basket.iterrows()})
mdl.bin_cap = pyo.Param(mdl.bins, initialize= {i:row["op_room_cap"] for i,row in op_df.iterrows()} )
mdl.dep_cap = pyo.Param(mdl.deps, initialize= {i:row["dep_cap"] for i,row in dep_df.iterrows()}, mutable=True)



# vars
mdl.X = pyo.Var(mdl.invs, mdl.bins, within=pyo.Binary)     # the amount from invoice i in bin j



### Objective ###

mdl.OBJ = pyo.Objective(expr=sum(mdl.X[i, b]*mdl.value[i] for 
                        i in mdl.invs for
                        b in mdl.bins), sense=pyo.maximize)


### constraints ###

# don't overstuff bin
def bin_limit(self, b):
    return sum(mdl.X[i, b]*mdl.weight[i] for i in mdl.invs) <= mdl.bin_cap[b]
mdl.bin_limit = pyo.Constraint(mdl.bins, rule=bin_limit)

# one_item can be only in a single op room.
def one_item(self, i,d):
    return sum(mdl.X[i,d,b] for b in mdl.bins) <=1
mdl.one_item = pyo.Constraint(mdl.invs, rule=one_item)



# department limits

mdl.dep_limits=pyo.ConstraintList()

for d in mdl.deps:
    d_list=[]
    for i in mdl.X:
        if d==i[1]:
            d_list.append(i)    
    mdl.dep_limits.add(expr=(sum(mdl.X[i]*mdl.weight[i[:2]] for i in d_list)<=mdl.dep_cap[d])) 



# solve it...
solver = pyo.SolverFactory('glpk')
results = solver.solve(mdl)

# save the output into dataframe



count=0
for i in mdl.X:
    if pyo.value(mdl.X[i])==1:
        count+=pyo.value(mdl.X[i])
        basket.loc[i[0],"op_room_id"]=i[2]



## Do not save the latest entry into the database, if the solution excludes previous entries from the allocation.
# Abort the entry and print the following      
if basket.shape[0]>count:
    ## Do not save it in database, abort operation and print the following
    print('Sorry,',basket.tail(1).index,', we cannot offer you a time slot on that day.')
    basket.drop(basket.tail(1).index, inplace=True)

else:
    #print on the secreen  
    basket.to_sql(name='today_op_list', if_exists='replace', index_label='id', con=cnx) #
    print(basket.tail(1), 'is registered.')
